In [1]:
import yaml
import clickhouse_connect
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import importlib
from ratelimit import limits, sleep_and_retry
import openai


In [1]:
from models import HF_LLMQuery, OpenAI_LLMQuery
import openai


### LLM inference with Question Decomposition

Need to provide configs for models and for prompting. 

Our prompting configs here definded as follows:

```yaml
PromptClass: QuestionTableRowsPrompt
prompt_configs:
  few_shot: true
  few_shot_text: |
    question: get people name with age equal 25 table: id, name, age
    First need to select all the people from table
    SELECT * FROM table
    Second need to select only people with age equal 25, that corresponds to the column age
    SELECT * FROM table WHERE age = 25
    A: SELECT * FROM table WHERE age = 25
  no_need_param: 20
  instruction_text: "Act as a proficent SQL developer and ask a question with a step by step reasoning"

```

That means 

- We are using Prompt Schema with Tables and Rows in out prompt. 

- We have specified that we use few shot and provided the text for the one-shot example.

- Also notice that we can pass a parameter of instruction, that will serve as "system" message. That could possibly increase generation quality for Large models with emergent abilities.

- We have passed a parameter that we do not need here, but this will not interrupt the correct programm execution as long as all the obligatory parameters are provided correctly.

In [3]:
LLM_QDecomp = HF_LLMQuery('configs/models/hf_model_config.yaml', 'configs/prompts/QDecomp.yaml')

query = 'How many active agency customers did we have on January 1st, 2022?'
tables = 'my_table'
rows = ['id', 'customer_action', 'date']

ans = LLM_QDecomp.query(query, tables=tables, rows = rows)
print(ans)

SELECT COUNT customer_action FROM table WHERE date = january 1st, 2022


### Zero-shot abilities

Now one can try a model with a zero-shot prompting.

Therefore configs for prompting are defined as follows

```yaml
PromptClass: QuestionTableRowsPrompt
prompt_configs:
  few_shot: false
  few_shot_text: "question: get people name with age equal 25 table: id, name, age \n SELECT name FROM table WHERE age = 25"
  no_need_param: 20
  instruction_text: ""

```

That means

- We again are using schema with tables and rows and should pass them as input
- no few-shot is used here, even if the text for it exists. The same would happen if we would set few_shot=true and the text will be empty.
- no system role instruction provided here
- we could see the non neccessary parameter again

In [17]:
LLM_Zero_shot = HF_LLMQuery('configs/models/hf_model_config.yaml', 'configs/prompts/Zero_shot.yaml')

query = 'How many active agency customers did we have on January 1st, 2022?'
tables = 'my_table'
rows = ['id', 'customer_action', 'date']

ans = LLM_Zero_shot.query(query, tables=tables, rows = rows)
print(ans)

SELECT COUNT customer_action FROM table WHERE date = january 1st, 2022


### Simpliest LLM Inference technique



In [15]:
LLM_Simple = HF_LLMQuery('configs/models/hf_model_config.yaml', 'configs/prompts/Simple_prompt.yaml')

query = 'How many active agency customers did we have on January 1st, 2022?'

ans = LLM_Simple.query(query)
print(ans)

SELECT COUNT active agency FROM table WHERE date = january 1st, 2022


In [26]:
ChatGPT = OpenAI_LLMQuery('configs/models/openai_model_config.yaml', 'configs/prompts/Simple_prompt.yaml')
query = 'How many active agency customers did we have on January 1st, 2022?'

try:
    ans = ChatGPT.query(query)
    print(ans)
except openai.error.RateLimitError:
    print('Invalid Token')

Invalid Token


In [7]:
filename = 'configs/database/db_credentials'
with open(f'{filename}.yaml','r') as f:
    output = yaml.safe_load(f)
print(output) 

{'host': 'db-access.improvado.io', 'port': 443, 'database': 'im_7690_4c9', 'user': 'im_7690_db', 'password': '4LFh3OgdbOC4HBxcXSsH'}


In [8]:
client = clickhouse_connect.get_client(**output)

In [9]:
all_tables = client.command('SHOW TABLES').split('\n')

In [22]:
output['model']

{'model_checkpoint': 'juierror/flan-t5-text2sql-with-schema'}

In [12]:
ans = querier.query('How many active agency customers did we have on January 1st, 2022?', tables=all_tables, rows = ['id', 'customer_action', 'date'])

Token indices sequence length is longer than the specified maximum sequence length for this model (3118 > 512). Running this sequence through the model will result in indexing errors


In [2]:
class QueryInterface:

    def __init__(self):
        '''
        Initialization of model and all arguments
        '''
        pass

    def query(self):
        '''
        query model with a question. extra arguments needed when using specific models
        '''
        pass

    def _init_model(self):
        '''
        Model initialization, with a path of configs
        '''
    def _init_prompter(self):
        '''
        Initialization of Prompt Changer class from config
        '''



class HF_LLMQuery(QueryInterface):

    def __init__(self, model_config_path, prompt_config_path, *args, **kwargs):
        self.model, self.tokenizer, self.generation_args = self._init_model(model_config_path)
        self.prompt_style = self._init_prompter(prompt_config_path)
        
    def query(self, question, **kwargs):

        prompt = self.prompt_style(question, **kwargs)
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
        
        output = self.model.generate(input_ids=input_ids, **self.generation_args)
        out_text = self.tokenizer.decode(output[0], skip_special_tokens=True)

        return out_text

    def _init_model(self, model_config_path):

        with open(f'{model_config_path}','r') as f:
            configs = yaml.safe_load(f)

        module = importlib.import_module('transformers')

        TokenizerClass = module.__getattr__(configs['TokenizerClass'])
        ModelClass = module.__getattr__(configs['ModelClass'])

        tokenizer = TokenizerClass.from_pretrained(**configs['tokenizer'])
        model = ModelClass.from_pretrained(**configs['model'])

        return model, tokenizer, configs['generation_args']

    def _init_prompter(self, prompt_config_path):

        with open(f'{prompt_config_path}','r') as f:
            configs = yaml.safe_load(f)

        module = importlib.import_module('prompting.hf_prompt_schemas')

        prompt_class = getattr(module, configs['PromptClass'])

        return prompt_class(**configs['prompt_configs'])
    

    

In [7]:
querier = HF_LLMQuery('configs/models/hf_model_config.yaml', 'configs/prompts/QDecomp.yaml')

In [8]:
ans = querier.query('How many active agency customers did we have on January 1st, 2022?', tables='my_table', rows = ['id', 'customer_action', 'date'])

In [9]:
ans

'A: SELECT COUNT customer_action FROM table WHERE date = january 1st, 2022'

In [10]:
querier.prompt_style('How many active agency customers did we have on January 1st, 2022?', tables='my_table', rows = ['id', 'customer_action', 'date'])

'question: get people name with age equal 25 table: id, name, age\nFirst need to select all the people from table\nSELECT * FROM table\nSecond need to select only people with age equal 25, that corresponds to the column age\nSELECT * FROM table WHERE age = 25\nA: SELECT * FROM table WHERE age = 25\n\nquestion: How many active agency customers did we have on January 1st, 2022? my_table: id, customer_action, date'

In [53]:
ans

'SELECT COUNT customer_action FROM table WHERE date = january 1st, 2022'

In [59]:
q =  """question: get people name with age equal 25 table: id, name, age
First need to select all the people from table
SELECT * FROM table
Second need to select only people with age equal 25, that corresponds to the column age
SELECT * FROM table WHERE age = 25
A: SELECT * FROM table WHERE age = 25"""

In [61]:
print(q)

question: get people name with age equal 25 table: id, name, age
First need to select all the people from table
SELECT * FROM table
Second need to select only people with age equal 25, that corresponds to the column age
SELECT * FROM table WHERE age = 25
A: SELECT * FROM table WHERE age = 25


In [57]:
query  = 'get people name with age equal 25'
tables = ['table']
rows = ['id', 'name', 'age']

res = """question: {} {}: {}""".format(
                                        query,
                                        ', '.join(tables),
                                        ', '.join(rows)
                                        )

In [58]:
res

'question: get people name with age equal 25 table: id, name, age'

In [40]:
', '.join(['table', 'table 2'])

'table, table 2'

In [3]:
querier = HF_LLMQuery('model_config.yaml', QuestionTableRowsPrompt)

In [5]:
ans = querier.query('get people name with age equal 25', tables='my_table', rows = ['id', 'name', 'age'])

/home/victor/anaconda3/lib/python3.7/site-packages/transformers/generation/utils.py:1392: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [6]:
querier.tokenizer.decode(ans[0], skip_special_tokens=True)

'SELECT name FROM table WHERE age = 25'

In [10]:
class HF_LLMQuery(QueryInterface):

    def __init__(self, model_config_path, prompt_config_path, *args, **kwargs):
        self.model, self.tokenizer, self.generation_args = self._init_model(model_config_path)
        self.prompt_style = self._init_prompter(prompt_config_path)
        
    def query(self, question, **kwargs):

        prompt = self.prompt_style(question, **kwargs)
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
        
        output = self.model.generate(input_ids=input_ids, **self.generation_args)
        out_text = self.tokenizer.decode(output[0], skip_special_tokens=True)

        return out_text

    def _init_model(self, model_config_path):

        with open(f'{model_config_path}','r') as f:
            configs = yaml.safe_load(f)

        module = importlib.import_module('transformers')

        TokenizerClass = module.__getattr__(configs['TokenizerClass'])
        ModelClass = module.__getattr__(configs['ModelClass'])

        tokenizer = TokenizerClass.from_pretrained(**configs['tokenizer'])
        model = ModelClass.from_pretrained(**configs['model'])

        return model, tokenizer, configs['generation_args']

    def _init_prompter(self, prompt_config_path):

        with open(f'{prompt_config_path}','r') as f:
            configs = yaml.safe_load(f)

        module = importlib.import_module('prompting.prompt_schemas')

        prompt_class = getattr(module, configs['PromptClass'])

        return prompt_class(**configs['prompt_configs'])
    


class OpenAI_LLMQuery(QueryInterface):

    def __init__(self, model_config_path, prompt_config_path, *args, **kwargs):
        self.model, self.generation_args = self._init_model(model_config_path)
        self.prompt_style = self._init_prompter(prompt_config_path)


    def query(self, question, **kwargs):

        prompt = self.prompt_style(question, **kwargs)
        out_text = self.get_answer(prompt)

        return out_text

    def _init_model(self, model_config_path):

        with open(f'{model_config_path}','r') as f:
            configs = yaml.safe_load(f)

        openai.api_key = configs['token']


        return openai.ChatCompletion, configs['generation_args']

    def _init_prompter(self, prompt_config_path):

        with open(f'{prompt_config_path}','r') as f:
            configs = yaml.safe_load(f)

        module = importlib.import_module('prompting.openai_prompt_schemas')

        prompt_class = getattr(module, configs['PromptClass'])

        return prompt_class(**configs['prompt_configs'])

    @sleep_and_retry
    @limits(calls=3, period=62)
    def get_answer(self, prompt: str) -> str:
        completion = self.model.create(
            messages = prompt,
            **self.generation_args
        )
        return str(completion["choices"][0]["message"]["content"])




In [16]:
querier = OpenAI_LLMQuery('configs/models/openai_model_config.yaml', 'configs/prompts/QDecomp.yaml')

In [17]:
ans = querier.query('How many active agency customers did we have on January 1st, 2022?', tables='my_table', rows = ['id', 'customer_action', 'date'])

RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [6]:
module = importlib.import_module('transformers')

In [7]:
AutoTokenizer = module.__getattr__('AutoTokenizer')

In [20]:
tokenizer = AutoTokenizer.from_pretrained("juierror/flan-t5-text2sql-with-schema")
model = AutoModelForSeq2SeqLM.from_pretrained("juierror/flan-t5-text2sql-with-schema")

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

KeyboardInterrupt: 

2992

In [51]:
query = 'SELECT * FROM country_region_mapping LIMIT 5'
res = client.command(query)


In [58]:
import signal

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Code execution timed out")

def run_with_timeout(func, timeout_seconds):
    # Set the signal handler for the timeout
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout_seconds)  # Set the timeout

    try:
        result = func()  # Execute the code
        signal.alarm(0)  # Reset the alarm if the code completes within the timeout
        return result
    except TimeoutException:
        print("Code execution timed out")
        return None

# Example usage
def long_running_code():
    # Your long-running code goes here
    query = 'SELECT * FROM web_analytics__utm_quality'
    res = client.command(query)
    return res

timeout = 1  # Set the desired timeout in seconds
result = run_with_timeout(long_running_code, timeout)

Code execution timed out


In [59]:
result

In [60]:
result

In [63]:
query = """SELECT column_name FROM information_schema.columns WHERE table_name = 'country_region_mapping' """
res = client.command(query)

In [65]:
res.split('\n')

['country_name', 'country_code', 'region_name', 'region_code']

In [4]:
query = 'SELECT *'
result = client.execute(query)

# Process the query result as needed
for row in result:
    print(row)

UnexpectedPacketFromServerError: Code: 102. Unexpected packet from server db-access.improvado.io:443 (expected Hello or Exception, got Unknown packet)